# **Movie Recommendation System**

-------------

## **Objective**

Recommender System is a system that seeks to predict or filter preferences according to the user's choices. Recommender systems are utilized in a variety of areas including movies, music, news, books, research articles, search queries, social tags, and products in general. Recommender systems produce a list of recommendations in any of the two ways-

Collaborative filtering: Collaborative filtering approaches build a model from the user's past behavior (le. Items purchased or searched by the user) as well as similar decisions made by other users. This model is then used to predict items (or ratings for items) that users may have an interest in.

Content-based filtering: Content-based filtering approaches uses a series of discrete characteristics of an item in order to recommend additional items with similar properties. Content-based filtering methods are totally based on a description of the item and a profile of the user's preferences. It recommends items based on the user's past preferences. Let's develop a basic recommendation system using Python and Pandas.

Let's develop a basic recommendation system by suggesting items that are most similar to a particular item, in this case, movies. It just tells what movies/items are most similar to the user's movie choice.

## **Data Source**

We are using a movie dataset from the YBI Foundation.

## **Import Library**

In [32]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib


## **Import Data**

In [33]:
# Load the dataset
df = pd.read_csv(r'https://raw.githubusercontent.com/YBIFoundation/Dataset/refs/heads/main/Movies%20Recommendation.csv')

# Display the first 5 rows
df.head()


,Movie_ID,Movie_Title,Movie_Genre,Movie_Language,Movie_Budget,Movie_Popularity,Movie_Release_Date,Movie_Revenue,Movie_Runtime,Movie_Vote,...,Movie_Homepage,Movie_Keywords,Movie_Overview,Movie_Production_House,Movie_Production_Country,Movie_Spoken_Language,Movie_Tagline,Movie_Cast,Movie_Crew,Movie_Director
0,1,Four Rooms,Crime Comedy,en,4000000,22.876230,09-12-1995,4300000,98.0,6.5,...,NaN,hotel new year's eve witch bet hotel room,It's Ted the Bellhop's first night on the job....,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,"[{'name': 'Allison Anders', 'gender': 1, 'depa...",Allison Anders
1,2,Star Wars,Adventure Action Science Fiction,en,11000000,126.393695,25-05-1977,775398007,121.0,8.1,...,http://www.starwars.com/films/star-wars-episod...,android galaxy hermit death star lightsaber,Princess Leia is captured and held hostage by ...,"[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twe...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,"[{'name': 'George Lucas', 'gender': 2, 'depart...",George Lucas
2,3,Finding Nemo,Animation Family,en,94000000,85.688789,30-05-2003,940335536,100.0,7.6,...,http://movies.disney.com/finding-nemo,father son relationship harbor underwater fish...,"Nemo, an adventurous young clownfish, is unexp...","[{""name"": ""Pixar Animation Studios"", ""id"": 3}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
3,4,Forrest Gump,Comedy Drama Romance,en,55000000,138.133331,06-07-1994,677945399,142.0,8.2,...,NaN,vietnam veteran hippie mentally disabled runni...,A man with a low IQ has accomplished great thi...,"[{""name"": ""Paramount Pictures"", ""id"": 4}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,"[{'name': 'Alan Silvestri', 'gender': 2, 'depa...",Robert Zemeckis
4,5,American Beauty,Drama,en,15000000,80.878605,15-09-1999,356296601,122.0,7.9,...,http://www.dreamworks.com/ab/,male nudity female nudity adultery midlife cri...,"Lester Burnham, a depressed suburban father in...","[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes


## **Describe Data**

In [34]:
# Get dataset information
df.info()

# Check dataset shape
print("Dataset shape:", df.shape)

# Display dataset columns
print("Columns:", df.columns)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4760 entries, 0 to 4759
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Movie_ID                  4760 non-null   int64  
 1   Movie_Title               4760 non-null   object 
 2   Movie_Genre               4760 non-null   object 
 3   Movie_Language            4760 non-null   object 
 4   Movie_Budget              4760 non-null   int64  
 5   Movie_Popularity          4760 non-null   float64
 6   Movie_Release_Date        4760 non-null   object 
 7   Movie_Revenue             4760 non-null   int64  
 8   Movie_Runtime             4758 non-null   float64
 9   Movie_Vote                4760 non-null   float64
 10  Movie_Vote_Count          4760 non-null   int64  
 11  Movie_Homepage            1699 non-null   object 
 12  Movie_Keywords            4373 non-null   object 
 13  Movie_Overview            4757 non-null   object 
 14  Movie_Pr

## **Data Visualization**

In [36]:
# Select important features for analysis
df_features = df[['Movie_Genre', 'Movie_Keywords', 'Movie_Tagline', 'Movie_Cast', 'Movie_Director']].fillna(' ')

# Combine selected features into a single string for each movie
x = df_features['Movie_Genre'] + ' ' + df_features['Movie_Keywords'] + ' ' + df_features['Movie_Tagline'] + ' ' + df_features['Movie_Cast'] + ' ' + df_features['Movie_Director']
print("Features shape:", x.shape)


Features shape: (4760,)


## **Data Preprocessing**

In [37]:
# Convert text data into numerical format using TF-IDF
Tfidf = TfidfVectorizer()
x = Tfidf.fit_transform(x)
print("TF-IDF Matrix shape:", x.shape)


TF-IDF Matrix shape: (4760, 17258)


## **Define Target Variable (y) and Feature Variables (X)**

In [38]:
# Compute cosine similarity
similarity_score = cosine_similarity(x)
print("Cosine Similarity Matrix shape:", similarity_score.shape)


Cosine Similarity Matrix shape: (4760, 4760)


## **Train Test Split**

In [39]:
# Define the recommendation process
Favourite_Movie_Name = input('Enter your favourite movie name: ')

# List of all movie titles
All_Movies_Title_List = df['Movie_Title'].tolist()

# Find the closest match for the input movie name
Movie_Recommendation = difflib.get_close_matches(Favourite_Movie_Name, All_Movies_Title_List)

if Movie_Recommendation:
    close_match = Movie_Recommendation[0]
    print("Closest match found:", close_match)

    # Get the index of the closest match
    Index_of_Close_Match_Movie = df[df.Movie_Title == close_match]['Movie_ID'].values[0]

    # Compute recommendation scores
    Recommendation_Score = list(enumerate(similarity_score[Index_of_Close_Match_Movie]))

    # Sort recommendations by similarity scores
    Sorted_Similar_Movies = sorted(Recommendation_Score, key=lambda x: x[1], reverse=True)

    # Display top 10 recommended movies
    print('Top 10 Movies Suggested for You:\n')
    i = 1
    for movie in Sorted_Similar_Movies:
        index = movie[0]
        title_from_index = df[df.Movie_ID == index]['Movie_Title'].values
        if i <= 10 and title_from_index:
            print(f"{i}. {title_from_index[0]}")
            i += 1
else:
    print("No close matches found. Please try again.")


Enter your favourite movie name: Avatar
Closest match found: Avatar
Top 10 Movies Suggested for You:

1. Avatar
2. The Girl on the Train
3. Act of Valor
4. Donnie Darko
5. Precious
6. Freaky Friday
7. The Opposite Sex
8. Heaven is for Real
9. Run Lola Run
10. Elizabethtown


## **Modeling**

No explicit machine learning model is used here as the recommendation is based on cosine similarity of textual features.

## **Model Evaluation**

Model evaluation in this context involves user feedback on the relevance of the recommendations provided.

## **Prediction**

Predictions are made based on the top-ranked movies from the similarity scores, and they are displayed to the user.

## **Explaination**



1. **Objective**:  
   The project aims to build a **movie recommendation system** that suggests movies similar to a user’s favorite movie. It relies on text-based data such as genres, keywords, cast, and director to find similarities between movies.

---

2. **Data Source**:  
   The dataset contains information about various movies, such as:
   - **Movie_Genre**: Categories like Action, Drama, Comedy, etc.
   - **Movie_Keywords**: Keywords that describe the movie.
   - **Movie_Tagline**: A tagline that summarizes the movie.
   - **Movie_Cast**: Actors starring in the movie.
   - **Movie_Director**: The director of the movie.

   This data is used to find patterns and similarities among movies.

---

3. **TF-IDF (Term Frequency-Inverse Document Frequency)**:  
   - TF-IDF is a technique that converts textual data into numerical values.  
   - It evaluates the importance of a word in a document relative to other documents in the dataset.  
   - For this project, it transforms features like **genre**, **keywords**, and **cast** into a numerical format so they can be compared mathematically.

---

4. **Cosine Similarity**:  
   - **Cosine Similarity** measures how similar two vectors are, irrespective of their magnitude.  
   - In this case, it calculates the similarity between movies based on their TF-IDF vectors.  
   - A higher similarity score indicates that the movies share many features and are more likely to appeal to the same audience.

---

5. **Output**:  
   - After calculating similarity scores for all movies, the system ranks them based on their similarity to the user’s input movie.  
   - The **top 10 similar movies** are displayed as recommendations to the user.  
   - The effectiveness of these suggestions can be validated based on user satisfaction.

This approach ensures the recommendations are tailored to the user's preferences by focusing on meaningful movie attributes.